# Documentação Técnica do Notebook: Extração de Dados de Biopsia de Mama - Fleury

## Objetivo Principal
**Este notebook realiza a extração, filtragem e persistência dos dados de exames de biópsia de mama do Fleury, com foco na detecção de casos de carcinoma.** O objetivo é identificar laudos que mencionam a topografia "MAMA" e, dentro desses, extrair frases associadas ao termo "carcinoma" para monitoramento e análise clínica.

## Tecnologias Utilizadas
- **PySpark**: Manipulação de dados em larga escala, consultas SQL, transformações e persistência em Delta Lake.
- **Databricks Feature Store**: Gerenciamento de features e tabelas Delta.
- **Octoops**: Monitoramento, alertas e integração com sistemas de controle de execução.
- **Delta Lake**: Persistência dos dados processados.
- **Python (logging, traceback, etc.)**: Controle de fluxo, tratamento de erros e registro de eventos.

## Fluxo de Trabalho/Etapas Principais
1. **Instalação de dependências**: Instalação do pacote `octoops` para monitoramento.
2. **Configuração do ambiente**: Reinicialização do kernel (Databricks) e importação das bibliotecas necessárias.
3. **Definição de filtros e tabelas**: Configuração dos filtros SQL para seleção incremental e extração dos dados relevantes.
4. **Consulta SQL principal**: Extração dos laudos que mencionam "MAMA" e "CARCINOMA", com uso de expressões regulares para identificar frases relevantes.
5. **Visualização dos dados extraídos**: Exibição dos resultados para validação.
6. **Persistência dos dados**: Salvamento dos dados processados na tabela Delta, com merge/upsert para garantir atualização incremental.
7. **Monitoramento e alertas**: Envio de alertas via Sentinel em caso de erros ou ausência de dados.

## Dados Envolvidos
- **Fonte**: Tabela `refined.saude_preventiva.fleury_laudos`.
- **Tabela de destino**: `refined.saude_preventiva.fleury_laudos_mama_biopsia_v2`.
- **Colunas importantes**:
    - `laudo_tratado`: Texto do laudo médico.
    - `RAW_CARCINOMA`: Frase extraída contendo "carcinoma" relacionada à "MAMA".
    - `HAS_CARCINOMA`: Indicador booleano da presença de carcinoma.
    - Identificadores: `ficha`, `id_item`, `id_subitem`, `id_cliente`, etc.

## Resultados/Saídas Esperadas
- DataFrame filtrado com laudos de biópsia de mama e indicação de presença de carcinoma.
- Persistência dos dados na tabela Delta para uso em análises clínicas e notificações.
- Alertas automáticos em caso de falhas ou ausência de dados.

## Pré-requisitos
- Ambiente Databricks ou Spark configurado.
- Pacotes: `octoops`, `databricks-feature-store`, `delta`.
- Acesso às tabelas Delta e permissões de escrita.

## Considerações Importantes
- O notebook utiliza expressões regulares avançadas para garantir a correta identificação de frases relevantes.
- O merge/upsert garante que os dados estejam sempre atualizados sem duplicidade.
- O monitoramento via Octoops/Sentinel facilita a rastreabilidade e resposta rápida a falhas.
- O filtro de idade e sexo garante que apenas pacientes do público-alvo sejam analisados.

---
A seguir, cada célula de código é precedida por uma explicação técnica detalhada sobre sua função e impacto no fluxo do notebook.

## Instalação do pacote octoops
Esta célula instala o pacote `octoops`, utilizado para monitoramento, alertas e integração com sistemas de controle de execução. O comando `%pip install` garante que o pacote esteja disponível no ambiente do notebook. Caso o ambiente já possua o pacote, a instalação será ignorada. O pacote é fundamental para o envio de alertas automáticos em etapas críticas do processamento.

In [ ]:
# %pip install databricks-feature-store -q
%pip install octoops


## Reinicialização do ambiente Python (Databricks)
Esta célula reinicia o kernel Python no ambiente Databricks, garantindo que todas as dependências recém-instaladas estejam disponíveis. O comando `dbutils.library.restartPython()` é específico do Databricks e não deve ser executado em ambientes fora dele. A reinicialização é útil após a instalação de novos pacotes para evitar conflitos de versões.

In [ ]:
dbutils.library.restartPython()

## Importação de bibliotecas
Esta célula importa todas as bibliotecas necessárias para o processamento dos dados:
- **PySpark**: Manipulação de DataFrames, funções SQL, tipos de dados e cálculos de datas.
- **Octoops**: Classes para monitoramento (`OctoOps`, `Sentinel`, `FeatureStoreManager`).
- **Databricks Feature Store**: Gerenciamento de tabelas Delta e features.
- **Logging, traceback, sys**: Controle de fluxo, registro de eventos e tratamento de erros.
Essas bibliotecas são essenciais para todas as etapas do notebook, desde a extração até a persistência e monitoramento dos dados.

In [ ]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel
from octoops import FeatureStoreManager
from databricks.feature_store import FeatureStoreClient

## Configuração do logger
Esta célula configura um logger Python para registrar mensagens de log durante a execução do notebook. O logger é utilizado para registrar informações, avisos e erros, facilitando o monitoramento e a depuração do fluxo. O objeto `logger` é criado com o nome do módulo atual (`__name__`) e será utilizado em funções de transformação e persistência.

In [ ]:
# O código configura um logger para registrar mensagens de log no módulo atual (__name__).
logger = logging.getLogger(__name__)

## Definição de filtros e tabelas
Esta célula define os filtros SQL e os nomes das tabelas Delta utilizadas no processamento:
- `table_biopsia`: Nome da tabela Delta de destino para persistência dos dados extraídos.
- `where_clause`: Filtro incremental para garantir que apenas novos dados sejam processados, comparando o campo `_datestamp` com o máximo já presente na tabela de destino.
- `filtro_extracao`: Filtro para selecionar apenas laudos de biópsia de mama, considerando siglas específicas de exames, sexo feminino e faixa etária de 40 a 75 anos.
Esses filtros garantem que apenas dados relevantes e novos sejam processados e persistidos.

In [ ]:
# filtros para a extração
table_biopsia = "refined.saude_preventiva.fleury_laudos_mama_biopsia_v2" 

where_clause = f"""
    WHERE
        _datestamp > (
            SELECT MAX(biop._datestamp)
            FROM {table_biopsia} biop
        )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado   = 'mama'
        AND sigla_exame IN (
            "BIOMAMUSCORE",
            "BIOMAMUSPAAF",
            "BIOMAMUS",
            "MAMOTOMIA",
            "MAMOTOMUS",
            "MAMCLIPE",
            "MAMOTORM"
        )
        AND UPPER(sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        ) 
"""

## Consulta SQL principal para extração de carcinoma
Esta célula constrói e executa a consulta SQL que extrai os laudos de biópsia de mama e identifica a presença de carcinoma:
- Utiliza `REGEXP_EXTRACT` para buscar a ocorrência da palavra "CARCINOMA" relacionada à "MAMA" no campo `laudo_tratado`.
- O padrão regex considera casos em que "CARCINOMA" aparece antes ou depois de "Topografia: MAMA".
- Cria a coluna `RAW_CARCINOMA` com a frase extraída e `HAS_CARCINOMA` como indicador booleano.
- Seleciona diversas colunas de identificação e características do laudo.
- Aplica os filtros incremental (`where_clause`) e de extração (`filtro_extracao`).
O resultado é um DataFrame com laudos filtrados e enriquecidos para análise clínica.

In [ ]:

# Utiliza REGEXP_EXTRACT para buscar, no texto do campo 'laudo_tratado', 
# a ocorrência da palavra "CARCINOMA" relacionada à "MAMA" (seja antes ou depois de "Topografia").
# O padrão regex considera maiúsculas/minúsculas e múltiplas linhas.
# Se encontrar, extrai "CARCINOMA" para a coluna RAW_CARCINOMA; caso contrário, retorna string vazia.
# O CASE seguinte verifica se RAW_CARCINOMA está vazia:
# - Se estiver vazia, HAS_CARCINOMA recebe FALSE (não há carcinoma relacionado à mama no laudo).
# - Se não estiver vazia, HAS_CARCINOMA recebe TRUE (há carcinoma relacionado à mama no laudo).
query = f"""
WITH base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        REGEXP_EXTRACT(
            flr.laudo_tratado,
            r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
            1
        ) AS RAW_CARCINOMA,
        CASE
            WHEN REGEXP_EXTRACT(
                    flr.laudo_tratado,
                    r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
                    1
                 ) = ''
            THEN FALSE
            ELSE TRUE
        END AS HAS_CARCINOMA
    FROM refined.saude_preventiva.fleury_laudos flr
    {where_clause}
)
SELECT *
FROM base
{filtro_extracao}
"""
df_spk_biopsia = spark.sql(query)
display(df_spk_biopsia)

## Visualização dos dados extraídos
Esta célula exibe o DataFrame resultante da consulta SQL anterior utilizando o comando `display`. Permite a inspeção visual dos dados extraídos, incluindo as colunas `RAW_CARCINOMA` e `HAS_CARCINOMA`, além dos identificadores dos laudos. É útil para validação dos resultados antes da persistência.

In [ ]:
display(df_spk_biopsia)

## Criação de tabela no Feature Store (comentado)
Esta célula contém código comentado para criação de uma tabela no Databricks Feature Store. O código define o nome da tabela, as chaves primárias (`ficha`, `id_item`, `id_subitem`), o schema baseado no DataFrame extraído e uma descrição detalhada. Caso seja necessário criar a tabela, basta descomentar e executar. O Feature Store facilita o gerenciamento de features para modelos de machine learning.

In [ ]:
# FeatureStore
# fs = FeatureStoreClient()
# fs.create_table(
#     name="refined.saude_preventiva.fleury_laudos_mama_biopsia",
#     primary_keys=["ficha",'id_item','id_subitem'],
#     schema=df_spk.schema,    
#     description="Jornada de Mama - Features extraídas de laudos de biopsia Fleury. Siglas: BIOMAMUSCORE, BIOMAMUSPAAF, BIOMAMUS, MAMOTOMIA, MAMOTOMUS, MAMCLIPE, MAMOTORM")



## Seleção de colunas e persistência dos dados
Esta célula realiza várias operações:
- Importa módulos para tratamento de erros e integração com Delta Lake.
- Define o caminho da tabela Delta de destino (`OUTPUT_DATA_PATH`).
- Seleciona as colunas de interesse do DataFrame extraído, incluindo identificadores, datas, sigla do exame, texto do laudo, indicadores de carcinoma, etc.
- Define a função `insert_data` para realizar merge/upsert dos dados na tabela Delta, garantindo atualização incremental sem duplicidade.
- Executa o salvamento dos dados se houver registros, imprimindo o total salvo. Caso não haja dados, envia alerta via Sentinel.
- Em caso de exceção, imprime o traceback e relança o erro.
Esta célula é responsável pela persistência final dos dados processados e pelo monitoramento de falhas.

In [ ]:
import sys
import traceback
from octoops import Sentinel
from delta.tables import DeltaTable

WEBHOOK_DS_AI_BUSINESS_STG = 'prd'


#OUTPUT_DATA_PATH = dbutils.widgets.get('OUTPUT_DATA_PATH')
OUTPUT_DATA_PATH = 'refined.saude_preventiva.fleury_laudos_mama_biopsia_v2'

# Selecionar colunas de interesse
df_spk = df_spk_biopsia.select("ficha","id_item","id_subitem","id_cliente","dth_pedido","dth_resultado", "sigla_exame","laudo_tratado","linha_cuidado","_datestamp","RAW_CARCINOMA","HAS_CARCINOMA").display()

# função para salvar dados na tabela
def insert_data(df_spk, output_data_path):  

    # Carrega a tabela Delta existente
    delta_table = DeltaTable.forName(spark, output_data_path)

    # Faz o merge (upsert)
    (delta_table.alias("target")
     .merge(
         df_spk.alias("source"),
         "target.ficha = source.ficha AND target.id_item = source.id_item AND target.id_subitem = source.id_subitem"
     )
     .whenMatchedUpdateAll()
     .whenNotMatchedInsertAll()
     .execute())      
try:
    if df_spk.count() > 0:
        
        #1/0 
        insert_data(df_spk, OUTPUT_DATA_PATH)
        # fs.write_table(
        #     name="refined.saude_preventiva.fleury_laudos_mama_biopsia",
        #     df=df_spk,
        #     mode="merge",
        # )
        print(f'Total de registros salvos na tabela: {df_spk.count()}')
            
    else:
        error_message = traceback.format_exc()
        summary_message = f"""Não há laudos para extração.\n{error_message}"""
        sentinela_ds_ai_business = Sentinel(
            project_name='Monitor_Linhas_Cuidado_Mama',
            env_type=WEBHOOK_DS_AI_BUSINESS_STG,
            task_title='Fleury Mama Biopsia'
        )

        sentinela_ds_ai_business.alerta_sentinela(
            categoria='Alerta', 
            mensagem=summary_message,
            job_id_descritivo='2_fleury_mama_biopsia'
        )
except Exception as e:
    traceback.print_exc()        
    raise e